# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125446e+02     1.540455e+00
 * time: 0.0912621021270752
     1     1.139582e+01     9.249075e-01
 * time: 0.9670670032501221
     2    -1.212056e+01     1.061574e+00
 * time: 1.0860710144042969
     3    -3.389027e+01     8.668200e-01
 * time: 1.2660911083221436
     4    -4.760557e+01     6.084612e-01
 * time: 1.4231090545654297
     5    -5.688078e+01     2.345721e-01
 * time: 1.567305088043213
     6    -5.978475e+01     1.394801e-01
 * time: 1.687053918838501
     7    -6.088967e+01     5.188590e-02
 * time: 1.8010640144348145
     8    -6.128716e+01     7.806815e-02
 * time: 1.9202580451965332
     9    -6.159980e+01     3.492794e-02
 * time: 2.0323619842529297
    10    -6.182671e+01     3.126537e-02
 * time: 2.152646064758301
    11    -6.200938e+01     2.452424e-02
 * time: 2.2649600505828857
    12    -6.207367e+01     1.745668e-02
 * time: 2.3843190670013428
    13    -6.213596e+01     1.557331e-02
 * time: 2.4964330196380

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522635
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666461615
